In [4]:
import os
from zwatershed import (partition_subvols,
                        eval_with_par_map,
                        stitch_and_save,
                        merge_by_thresh)
datadir = "/Users/michielk/oxdata/P01/EM/Myrf_01/SET-B/B-NT-S10-2f_ROI_00"
# pred_file = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs1_eed2_main.h5')  # dataset: 'main'
pred_file = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_main_vol01.h5')  # dataset: 'main'
out_folder = os.path.join(datadir, 'zws_vol01_')
outname = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_zws.h5')  # dataset: 'main'
max_len = 300


In [5]:
partition_data = partition_subvols(pred_file,out_folder,max_len)


dims [ 184 1188 1204]
num_vols [1 4 5]
deltas [184 297 240]


In [ ]:
NUM_WORKERS=5
# eval_with_spark(partition_data[0])
eval_with_par_map(partition_data[0], NUM_WORKERS)


In [3]:
stitch_and_save(partition_data, outname)


i,x,y,z 0 0 0 0
i,x,y,z 1 0 0 1
i,x,y,z 2 0 0 2
i,x,y,z 3 0 0 3
i,x,y,z 4 0 0 4


IOError: Unable to open file (Unable to open file: name = '/users/michielk/oxdata/p01/em/myrf_01/set-b/b-nt-s10-2f_roi_00/zws_vol010_294_0_vol/basic.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
%matplotlib nbagg

import numpy as np
import sys
import time
import os
import h5py
import os.path as op
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from multiprocessing import Pool
from itertools import product
from zwatershed import (partition_subvols,
                        eval_with_par_map,
                        stitch_and_save,
                        merge_by_thresh)
# from par_funcs import *
# sys.path.append('..')
cmap = matplotlib.colors.ListedColormap(np.vstack( ((0, 0, 0), np.random.rand(1e6, 3))) )
V = 20

datadir = "/Users/michielk/oxdata/P01/EM/Myrf_01/SET-B/B-NT-S10-2f_ROI_00"
zwsbase = os.path.join(datadir, "zws")
outname = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_zws.h5')  # dataset: 'main'

num,thresh = 0,2000

plt.subplot(1,2,1)
basic_file = h5py.File(zwsbase + '0_0_0_vol/'+'basic.h5','r')
seg_init = np.array(basic_file['seg'])
rg_init = np.array(basic_file['rg'])
keeps = rg_init[:,0]<rg_init[:,1]
rg_init = rg_init[keeps,:]

seg_sizes_init = np.array(basic_file['counts'])
basic_file.close()
plt.imshow(seg_init[V,:,:], cmap=cmap)
plt.title('seg_init')


In [ ]:
plt.subplot(1,2,2)
f = h5py.File(outname, 'a')
s,e = f['starts'][num],f['ends'][num]
seg = f['seg'][s[0]:e[0]-3,s[1]:e[1]-3,s[2]:e[2]-3]
seg_sizes = np.array(f['seg_sizes'])
rg = np.array(f['rg_'+str(num)])
f.close()
plt.imshow(seg[V,:,:], cmap=cmap)
plt.title('seg_after_stitching')
plt.show()

print "num_segs",len(np.unique(seg_init)),len(np.unique(seg))
print "rg lens",len(rg_init),len(rg)


In [ ]:
# seg_init_merged = merge_by_thresh(seg_init,seg_sizes_init,rg_init,thresh)
seg_merged = merge_by_thresh(seg,seg_sizes,rg,thresh)

plt.subplot(1,2,1)
plt.imshow(seg_init_merged[V,:,:], cmap=cmap)
plt.title('merged init')
plt.subplot(1,2,2)
plt.imshow(seg_merged[V,:,:], cmap=cmap)
plt.title('merged')
plt.show()

print "num_segs",len(np.unique(seg_init)),len(np.unique(seg))
print "rg lens",len(rg_init),len(rg)
